<a href="https://colab.research.google.com/github/IlyasNasirov/notebook/blob/main/11_MA%CA%BCLUMOTLARNI_TAYYORLASH_11_28_YAKUNIY_AMALIY_VAZIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/h4PPC71.png)
#11-modul. Maʼlumotlarga ishlov berish. Maʼlumotlarni tayyorlash
# 11.28. Modul boʻyicha yakuniy amaliy vazifa

**Bu vazifalarni bajargach, maʼlumotlarni yuklash, tozalash, oʻzgartirish, matnlarni normalizatsiya qilish, qiymatlarni tekshirish, guruhlash va tahlil qilish kabi maʼlumotlarga ishlov berish (data processing) koʻnikmalarini egallaysiz.**

#**Umumiy maʼlumotlar bilan ishlash**

#Dataset haqida qisqacha maʼlumot
Restoran savdosi boʻyicha maʼlumotlar toʻplami 17 534 tranzaksiya uchun maʼlumotlarni oʻz ichiga oladi. Maʼlumotlar yetishmayotgan yoki baʼzilari toʻliq kiritilmagan boʻlishi mumkin. Maʼlumotlar toʻplamida asosiy taomlar, shirinliklar, ichimliklar kabi bir nechta toifadagi savdo tafsilotlari mavjud.


 0.   **Unnamed 0** – indekslarni;
 1.   **Order ID** – buyurtma IDʼsini;
 2.   **Customer ID** – mijoz IDʼsini;
 3.   **Category** – mahsulotlar kategoriyasini;
 4.   **Item** – mahsulot nomini;
 5.   **Price** – mahsulotning narxini;
 6.   **Quantity** – qancha mahsulot sotilgani miqdorini;
 7.   **Order Total** – umumiy narx qancha ekanini;
 8.   **Order Date** – buyurtma berilgan sanani (vaqt);
 9.   **Payment Method** – toʻlov usuli turini saqlaydi.

#1) Quyida berilgan datasetni yuklab olib, ustun nomlarini koʻring va ularni maʼno jihatdan tushunarli shaklga keltiring. `Unnamed: 0` ustunini tashlab yuboring!

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/alishermutalov/praktikum-datasets/refs/heads/praktikum/restaurant_sales.csv"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


#2) Maʼlumotlarning umumiy holatini (info(), describe()) tekshiring.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        17534 non-null  object 
 1   Customer ID     17534 non-null  object 
 2   Category        17498 non-null  object 
 3   Item            15776 non-null  object 
 4   Price           17534 non-null  float64
 5   Quantity        17104 non-null  float64
 6   Order Total     17104 non-null  float64
 7   Order Date      17534 non-null  object 
 8   Payment Method  16452 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB


In [6]:
df.describe()

,Price,Quantity,Order Total
count,17534.000000,17104.000000,17104.000000
mean,6.207312,3.014149,19.914494
std,4.993782,1.414598,18.732549
min,-1.000000,1.000000,1.000000
25%,3.000000,2.000000,7.500000
50%,5.000000,3.000000,15.000000
75%,7.000000,4.000000,25.000000
max,20.000000,5.000000,100.000000


#3) Har bir ustundagi boʻsh qiymatlarni aniqlang va ularni toʻldirish strategiyasini ishlab chiqing.

In [32]:
df.head(8)

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,NaN,side salad#?,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,-1.0,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash
5,ORD_381680,CUST_031,Main Dishes,Salmon,18.0,5.0,90.0,2022-07-20,Digital Wallet
6,ORD_270994,CUST_071,Side Dishes,Garlic Bread,4.0,5.0,20.0,2022-08-19,Credit Card
7,ORD_146656,CUST_077,Main Dishes,NaN,15.0,3.0,45.0,2023-02-15,Cash


In [52]:
df.isna().sum()
# "Item" ustunni bosh bolgan qiymatlrni unga togri keladigan "Category" boyicha toldiramiz.
# "Category" ustuni teskarisi "Item" ga qarab tuldirishimiz mumkin
# "Quantity" ustunini "Price" va 'Order Total' ga qarab topishimiz mumkin
# "Order Total" ham shunga ohshash, yaniyki  "Price" va "Quatitiy"ni kopaytiramiz
# "Payment Method" eng kop uchraydigan bilan toldirsak boladi

,0
Order ID,0
Customer ID,0
Category,36
Item,1758
Price,0
Quantity,430
Order Total,430
Order Date,0
Payment Method,1082


In [51]:
df[df['Price']==-1.0]['Order ID'].count()

np.int64(876)


#4) Har bir ustundagi takroriy qiymatlarni toping va ularga qanday oʻzgartirish kiritish kerakligini hal qiling.

In [ ]:
#javobni shu yerda yozing

#5) Maʼlumotlar toʻplamidagi noto‘g‘ri maʼlumotlar (masalan, son turishi kerak bo‘lgan ustunda matn bo‘lsa) mavjudligini tekshiring va to‘g‘rilang.

In [ ]:
#javobni shu yerda yozing

#6) Maʼlumotlar toʻplamidagi **katta/kichik harf** muammolari boʻlgan joylarni toping va bir xil formatga keltiring.

In [ ]:
#javobni shu yerda yozing

#7) Maʼlumotlar turini oʻzgartirish.
 * Ustunlarning maʼlumot turlarini tekshiring. Notoʻgʻri turlarga ega boʻlgan ustunlarni toʻgʻri formatga oʻtkazing.
 * `order_date` yoki unga oʻxshash ustunni datetime formatiga oʻtkazing.

In [ ]:
#javobni shu yerda yozing

#8) Mahsulot nomlarida noto‘g‘ri yoki nomaʼlum belgilar (?, NaN, #) mavjudligini tekshiring va ularni tuzating.

In [ ]:
#javobni shu yerda yozing

#**Maʼlumotlarni filtrlash va qayta ishlash**

#9) Manfiy narx yoki 0 ga teng narx qiymatlarini toping va ularni qanday oʻzgartirish kerakligini hal qiling.

In [ ]:
#javobni shu yerda yozing

#10) Boʻsh toifalar (kategoriya) mavjudligini tekshiring va toʻgʻri kategoriya berish strategiyasini ishlab chiqing.

In [ ]:
#javobni shu yerda yozing

#11) Bir xil, lekin turlicha yozilgan qiymatlar (Coffee, cofee, COFEE) mavjudligini tekshiring va tuzating.

In [ ]:
#javobni shu yerda yozing

#12) `Customer ID` yoki boshqa identifikatorlarning takrorlanishini tekshiring.

In [ ]:
#javobni shu yerda yozing

#13) Mahsulotning ortiqcha taʼriflarini olib tashlang (Water (0.5), Water (1.0), Water → faqat Water qoldirish).

In [ ]:
#javobni shu yerda yozing

#**Qayta ishlangan maʼlumotlar bilan tahlil**

#14) Eng koʻp sotilgan 5 ta mahsulotni aniqlang.

In [ ]:
#javobni shu yerda yozing

#15) Har bir mijoz oʻrtacha qancha pul sarflaganini hisoblang.

In [ ]:
#javobni shu yerda yozing

#16) Mijozlarning qaysi kunlarda koʻproq xarid qilgani bo‘yicha tahlil qiling.

In [ ]:
#javobni shu yerda yozing

#17) Narxi oʻrtacha narxdan ikki barobar yuqori boʻlgan mahsulotlarni toping.

In [ ]:
#javobni shu yerda yozing

#18) Mahsulotlar kategoriyalari bo‘yicha tahlil qiling va eng ko‘p sotiladigan kategoriyalarni aniqlang.

In [ ]:
#javobni shu yerda yozing

#19) Har bir mijozning oʻrtacha xarid miqdorini (`groupby("customer_id")`) hisoblang.

In [ ]:
#javobni shu yerda yozing

#20) Qanday omillar sotuv hajmiga ta’sir qiladi? Bu bo‘yicha umumiy xulosa chiqaring. Javobni quyida kommentariya sifatida yozing!

In [ ]:
#javobni shu yerda yozing